# Introduction to MPI - Day 2

## 1. Collective Communication

In the previous program, the local residuals were calculated on each MPI process and reduced to the root process (the rank 0 process) by reduction operation `MPI_SUM`.
```cpp
double tot_res;
residual  = local_L2_residual(ptr_rows, mesh_size, space, &submesh[0][0], &subrhs[0][0]);
MPI_Reduce(&residual, &tot_res, 1, MPI_DOUBLE, MPI_SUM, 0, world);
```

It is recommended but not required from MPI Standard 4.0 that the same order is maintained for the reduction operation whenever the same arguments are applied to the function. For operations that a specific ordering is significant, one may consider use `MPI_Gather` to collect data to a single process before a local reduction operation.


**`TODO`**: In [lapalce_mpi_collective.c](./laplace_mpi_collective.c), replace the `MPI_Reduce` function with `MPI_Gather` and sum up residuals on the root process. Once you are finished run the next cell. If you are stuck, peek the solution at [soln_laplace_mpi_collective.c](./solution/laplace_mpi_collective.c)

In [2]:
!cd ./solution && make clean && make collective && echo "Compilation Successful!" && mpiexec -np 8 ./laplace_mpi_collective 100 500 Jacobi > convergence_collective.txt
!cd ./solution && tail -20 convergence_collective.txt 

rm -f *.o laplace_mpi_win
mpicc -g -Wall -O3 -lm -o laplace_mpi_collective laplace_mpi_collective.c mesh.c solver.c
laplace_mpi_collective.c: In function ‘main’:
laplace_mpi_collective.c:159:21: warning: unused variable ‘tot_res’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
  159 |     double residual,tot_res;
      |                     ^~~~~~~
laplace_mpi_collective.c:147:5: warning: unused variable ‘index’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
  147 | int index, top_flag, bottom_flag;
      |     ^~~~~
Compilation Successful!
Residual1  0.075272
Residual1  0.075234
Residual1  0.075196
Residual1  0.075158
Residual1  0.075120
Residual1  0.075082
Residual1  0.075045
Residual1  0.075007
Residual1  0.074969
Residual1  0.074931
Residual1  0.074894
Residual1  0.074856
Residual1  0.074818
Residual1  0.074780
Residual1  0.074743
Residual1  0.074705
Residual

## 2. One-sided Communication

The mechanism of one-sided communication (aka remote memory access) proposes an entirely different comunication pattern than the message-passing examples that we have seen. There are no senders and receivers involved in this pattern, meaning, messages do not need to match before the transfer. Instead, by creating a memory object called window at the target in which the memory window is accessed, the origin, in which the call is performed, can put data to and get data from that window followed by whatever local load/store operations.

In our example [laplace_mpi_win.c](./laplace_mpi_win.c), each MPI process is used as both the origin and the target - when the window is created and exposed, it is treated as the target, when an RMA call is performed, it is the origin. 

`As the target`, two window objects are created with separate allocated memory in an MPI process, one for the top row of ghost nodes, the other for the bottom row of ghost nodes.
```cpp
MPI_Win upper_win, lower_win;
double *upper_bnd, *lower_bnd;
/* win for the top row of ghost nodes */
MPI_Alloc_mem(sizeof(double )*mesh_size, MPI_INFO_NULL, &upper_bnd);
MPI_Win_create(upper_bnd, mesh_size *sizeof(double), sizeof(double),\
               MPI_INFO_NULL, MPI_COMM_WORLD, &upper_win);

/* win for the bottom row of ghost nodes */
MPI_Alloc_mem(sizeof(double )*mesh_size, MPI_INFO_NULL, &lower_bnd);
MPI_Win_create(lower_bnd, mesh_size *sizeof(double), sizeof(double),\
               MPI_INFO_NULL, MPI_COMM_WORLD, &lower_win); 
```

`As the origin`, the MPI process is very much like the sender doing `MPI_Send`, it needs to transfer out the top and bottom full nodes for the neighbouring processes to update their ghost nodes, however, this is done by using `MPI_Put`.

```cpp
/* put the bottom row of full nodes to the rank below */
MPI_Put(submesh[1], mesh_size, MPI_DOUBLE, lower, 0, mesh_size, MPI_DOUBLE, upper_win);

/* put the top row of full nodes to the rank above */
MPI_Put(submesh[*ptr_rows - 2], mesh_size, MPI_DOUBLE, upper, 0, mesh_size, MPI_DOUBLE, lower_win);   
```

Next, after completing the RMA operation the same MPI process needs to load/store data from its window object to update its own ghost nodes. This makes it the `target`.

```cpp
/* update the full bottom and top of submesh */
if (rank > 0 && rank < (cells -1)){
    for (int i=0; i< mesh_size; i++){
        submesh[0][i] = lower_bnd[i]; /* bottom */
        submesh[*ptr_rows-1][i] = upper_bnd[i]; /* top */            
}
}
else if (rank ==0 ){ /* bottom rank only updates its top full row */
    for (int i = 0; i< mesh_size; i++ ){

        submesh[*ptr_rows-1][i] = upper_bnd[i];
    }
}

else if (rank == (cells -1) ){ /* top rank only updates its bottom full row */ 
    for (int i = 0; i <mesh_size; i++){
        submesh[0][i] = lower_bnd[i];
    }
}
```

Notice that the transition for this MPI proceess 
$$Origin \to Target \to Origin$$
where each phase is concluded by a sychronisation call `MPI_Win_fence`.

Again, the window object doesn't have to match with a RMA operation `MPI_Put`. In contrast, it is designed to be exposed by multiple meessages from any process in the communicator group.

**`TODO`**: Replace the two window objects in [laplace_mpi_win.c](./solution/laplace_mpi_win.c) with a single window object. Use this window object to host both top and bottom ghost nodes for the neighbouring processes' access. Once you are finished run the next cell. If you are stuck, peek the solution at [soln_laplace_mpi_win.c](./solution/laplace_mpi_win.c)

In [3]:
!cd ./solution && make clean && make win && echo "Compilation Successful!" && mpiexec -np 8 ./laplace_mpi_win 100 500 Jacobi > convergence_win.txt
!cd ./solution && tail -20 convergence_win.txt 

rm -f *.o laplace_mpi_win
mpicc -g -Wall -O3 -lm -o laplace_mpi_win laplace_mpi_win.c mesh.c solver.c
Compilation Successful!
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
Residual1  0.075310
Residual1  0.075272
Residual1  0.075234
Residual1  0.075196
Residual1  0.075158
Residual1  0.075120
Residual1  0.075082
Residual1  0.075045
Residual1  0.075007
Residual1  0.074969
Residual1  0.074931
Residual1  0.074894
Residual1  0.074856
Residual1  0.074818
Residual1  0.074780
Residual1  0.074743
Residual1  0.074705
Residual1  0.074668
Residual1  0.074630
Residual1  0.077502


## 3. MPI-IO

After the iterative method is terminated with an acceptable magnitude of the error, we want to write the approximation over the unit square domain for the visualisation, validation and whatnots. The question now is the following: each MPI process only possesses a subdomain so how to merge the data to a common file?

Traditionally, one may either send all the subdomains to a root process, such is called master-slave communication mode, or output each individual subdomains for a different program to scan and combine.

MPI-IO provides the ability for multiple MPI processes to write into a common file. The basic MPI-IO procedures resembles with the POSIX IO:
```cpp
/* Open a file */
MPI_File_open(MPI_COMM_WORLD, file_name, MPI_MODE_CREATE|MPI_MODE_WRONLY, MPI_INFO_NULL, &fp);

/* Write to a file */
MPI_File_write_at(fp, offset, &submesh[1][0], count, MPI_DOUBLE, &IO_status );

/* Close a file */
MPI_File_close(&fp);

```
Each process starts to write at the offset position of the common file. It is the programmer's responsibility to ensure the offset is set correctly.

**`TODO`**: Completes the output routine for middle rank MPI processes in [laplace_mpi_io.c](./solution/laplace_mpi_io.c). Once you are finished run the next cell. If you are stuck, peek the solution at [soln_laplace_mpi_io.c](./solution/laplace_mpi_io.c)

In [4]:
!cd ./solution && make clean && make io && echo "Compilation Successful!" && mpiexec -np 4 ./laplace_mpi_io 27 500 Jacobi > convergence_io.txt
!cd ./solution && tail -20 convergence_io.txt 
!cat laplace-soln-whole

rm -f *.o laplace_mpi_win
mpicc -g -Wall -O3 -lm -o laplace_mpi_io laplace_mpi_io.c mesh.c solver.c
laplace_mpi_io.c: In function ‘main’:
laplace_mpi_io.c:226:12: warning: unused variable ‘w_count’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
  226 | int count, w_count ;
      |            ^~~~~~~
Compilation Successful!
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
[WARNING] yaksa: 1 leaked handle pool objects
Residual  0.010330 after iter 482 
Residual  0.010255 after iter 483 
Residual  0.010180 after iter 484 
Residual  0.010106 after iter 485 
Residual  0.010032 after iter 486 
Residual  0.009959 after iter 487 
Residual  0.009886 after iter 488 
Residual  0.009814 after iter 489 
Residual  0.009743 after iter 490 
Residual  0.009672 after iter 491 
Residual  0.009601 after iter 492 
Residual  0.009531 after iter 493 
Residual

MPI-IO only supports unformatted binary files, so you will need to interpret the unformatted data by whichever tools you like. (MATLAB is my go-to for this task.)

## 4. Profile with mpiP
There are many profilers available for profiling MPI codes. We use mpiP in this workshop in favour of its simplicity, light weight and portability as it is built on PMPI instrumentation.